In [1]:
import pandas as pd

In [2]:
EXERCISE = 2
df = pd.read_csv(f'./data/Exercise_{EXERCISE}_valid.csv')
df.ID = df.ID.astype(int)
df.TrialInfo = df.TrialInfo.astype(int)
df.Repetitions = df.Repetitions.astype(float)

In [3]:
df_mean = df.groupby(["ID", "TrialInfo"]).agg(
    Mean_Repetitions=("Repetitions", "mean"),
    Group=("Group", "first"),
    Gender=("Gender", "first")
).reset_index()

#sort by ID and TrialInfo
df_mean = df_mean.sort_values(by=["ID", "TrialInfo", "Gender"])

In [4]:
# Only keep TrialInfo 1 and 4
df_filtered = df_mean[df_mean['TrialInfo'].isin([1, 4])]

# Pivot to wide format: one row = one student
df_pivot = df_filtered.pivot_table(index=['ID', 'Group', "Gender"],
                                   columns='TrialInfo',
                                   values='Mean_Repetitions').reset_index()

# Rename columns for easier handling
df_pivot = df_pivot.rename(columns={1: 'Week1', 4: 'Week4'})

print(df_pivot.head())

TrialInfo  ID Group Gender     Week1  Week4
0           1    VR      w  1.250000    1.0
1           2    VR      w  1.333333    1.6
2           4    VR      w  1.400000    1.0
3           6    VR      m  1.000000    1.0
4           8    VR      m  1.000000    1.4


In [5]:
import json
from scipy import stats

results = {}

genders = df_pivot["Gender"].unique()
groups = df_pivot["Group"].unique()

for group in groups:
    group_data = df_pivot[df_pivot['Group'] == group]

    week1 = group_data['Week1']
    week4 = group_data['Week4']

    stat, p_normality = stats.shapiro(week1 - week4)

    if p_normality > 0.05:
        t_stat, p_value = stats.ttest_rel(week1, week4)
        test_used = 'Paired t-test'
    else:
        w_stat, p_value = stats.wilcoxon(week1, week4)
        test_used = 'Wilcoxon signed-rank'

    significant = bool(p_value < 0.05)

    results[group] = {
        'test': test_used,
        'p_value': float(p_value),
        'significant': significant,
        "p-normality": float(p_normality),
        "gender": "gesamt",
        "size": len(week1)
    }

    for gender in genders:
        gender_data = group_data[group_data["Gender"] == gender]
        week1 = gender_data['Week1']
        week4 = gender_data['Week4']

        stat, p_normality = stats.shapiro(week1 - week4)

        if p_normality > 0.05:
            t_stat, p_value = stats.ttest_rel(week1, week4)
            test_used = 'Paired t-test'
        else:
            w_stat, p_value = stats.wilcoxon(week1, week4)
            test_used = 'Wilcoxon signed-rank'

        significant = bool(p_value < 0.05)

        results[f"{group}-{gender}"] = {
            'test': test_used,
            'p_value': float(p_value),
            'significant': significant,
            "p-normality": float(p_normality),
            "gender": gender,
            "size": len(week1)
        }

# Save results to JSON
with open(f'./results/significancy_{EXERCISE}.json', 'w') as f:
    json.dump(results, f, indent=4)
